In [7]:
import os
import pandas as pd 
import librosa
import numpy as np
import matplotlib.pyplot as plt
import torch 
from IPython.display import Audio
from tqdm import tqdm

import nibabel as nib
from nilearn import plotting
from nilearn.glm.first_level import FirstLevelModel
from nilearn.plotting import plot_design_matrix
from nilearn.plotting import plot_contrast_matrix

In [2]:
fmri_music_path = '/data01/data/fMRI_music_genre'
print(os.listdir(fmri_music_path))
stimuli_path = fmri_music_path + '/data_wav'
print(os.listdir(stimuli_path))

neural_path = fmri_music_path + '/ds003720/sub-001'
print(os.listdir(neural_path))
# fmri_data = nib.load(event_path_train.replace('events.tsv', 'bold.nii'))

['data_wav', 'ds003720', 'glmsingle_outputs']
['brain2music-captions.csv', 'genres_original', '.DS_Store', 'features_30_sec.csv', 'images_original', 'features_3_sec.csv']
['anat', 'func']


In [5]:
def is_training_events_file(filename, phase):
    return filename.startswith("sub-001_task-"+phase+"_run-") and filename.endswith("_events.tsv")

all_files = os.listdir(neural_path + '/func')
training_files = [filename for filename in all_files if is_training_events_file(filename, "Training")]
print(training_files)

['sub-001_task-Training_run-01_events.tsv', 'sub-001_task-Training_run-02_events.tsv', 'sub-001_task-Training_run-03_events.tsv', 'sub-001_task-Training_run-04_events.tsv', 'sub-001_task-Training_run-05_events.tsv', 'sub-001_task-Training_run-06_events.tsv', 'sub-001_task-Training_run-07_events.tsv', 'sub-001_task-Training_run-08_events.tsv', 'sub-001_task-Training_run-09_events.tsv', 'sub-001_task-Training_run-10_events.tsv', 'sub-001_task-Training_run-11_events.tsv', 'sub-001_task-Training_run-12_events.tsv']


In [8]:
def get_fmri_data(training_files, t_r):
    eff_map_arrays = []
    for event in tqdm(training_files):
        event_path = neural_path + '/func/' + event
        events_df = pd.read_csv(event_path, sep='\t')
        fmri_img = nib.load(event_path.replace('events.tsv', 'bold.nii'))

        events_df['trial_type'] = events_df['genre'].str.strip("'") + ' - ' + events_df['track'].astype(str)
        first_level_model = FirstLevelModel(t_r)
        first_level_model = first_level_model.fit(fmri_img, events=events_df)
        design_matrix_gentrack = first_level_model.design_matrices_[0]

        contrast_val_gentrack = np.eye(design_matrix_gentrack.shape[1])[0:-13]
        eff_map_gentrack = first_level_model.compute_contrast(contrast_val_gentrack, output_type="effect_size")
        data_gentrack = eff_map_gentrack.get_fdata()
        eff_map_arrays.append(data_gentrack)

    eff_map_stacked = np.concatenate(eff_map_arrays, axis=-1)
    return eff_map_stacked

eff_map_stacked = get_fmri_data(training_files, 1.5)
print(eff_map_stacked.shape)


  0%|          | 0/12 [00:00<?, ?it/s]

/home/matteoc/miniconda3/envs/speech-meg/lib/python3.9/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: genre, end, track, start
  warnings.warn(
  8%|▊         | 1/12 [00:30<05:37, 30.67s/it]/home/matteoc/miniconda3/envs/speech-meg/lib/python3.9/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: genre, end, track, start
  warnings.warn(
 17%|█▋        | 2/12 [01:01<05:05, 30.59s/it]/home/matteoc/miniconda3/envs/speech-meg/lib/python3.9/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: genre, end, track, start
  warnings.warn(
 25%|██▌       | 3/12 [01:34<04:46, 31.82s/it]/home/matteoc/miniconda3/envs/speech-meg/lib/python3.9/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarnin

(96, 96, 68, 492)


In [18]:
event_path = neural_path + '/func/' + training_files[0]
events_df = pd.read_csv(event_path, sep='\t')
genre = events_df['genre'].iloc[0].strip("'") 
print(genre)
track = int(events_df['track'].iloc[0])
print(track)
start = float(events_df['start'].iloc[0])
print(start)
end = float(events_df['end'].iloc[0])
print(end)
track_str = str(track).zfill(5)  
print(track_str)

disco
9
4.21
19.21
00009


In [19]:
# def construct_wav_path(row):
#     genre = row['genre'].strip("'")  
#     track = int(row['track'])
#     track_str = str(track).zfill(5)  
#     return os.path.join(base_directory, genre, f"{genre}.{track_str}.wav")